# BBC news Data fetch and save

This notebook downloads all the data from the reuters about a specific topic and saves it to the database

In [2]:
import bs4
from bs4 import BeautifulSoup
import requests
import ray
from math import ceil
import time
from datetime import datetime
from http.client import HTTPSConnection
from urllib.parse import urljoin, unquote
import re
from time import sleep
import datetime
import asyncio
import aiohttp
import os.path
from dateutil import parser

In [3]:
import duckdb

class NewsMetaRepository:
    def __init__(self, csv_file=None):
        self.connection = duckdb.connect(database=':memory:', read_only=False)
        if os.path.isfile(f'{csv_file}/news_meta.csv'):
            try:
                self.connection.execute(f"IMPORT DATABASE '{csv_file}';")
            except Exception as e:
                self._create_table()
                self.connection.execute(f"EXPORT DATABASE '{csv_file}';")
        else:
            self._create_table()

    def _create_table(self):
        self.connection.execute("CREATE TABLE news_meta (id VARCHAR, title VARCHAR UNIQUE, url VARCHAR, timestamp VARCHAR UNIQUE,term VARCHAR)")
        self.connection.execute("CREATE SEQUENCE id_sequence START 1 INCREMENT BY 1;")

    def insert(self, news_meta):
        self.connection.execute("PREPARE insert_meta AS "
                                "INSERT INTO news_meta VALUES (nextval('id_sequence'), ?, ?, ?, ?) ON CONFLICT DO NOTHING;")
        self.connection.execute(f"EXECUTE insert_meta('{news_meta['title']}', '{news_meta['url']}', '{news_meta['timestamp']}', '{news_meta['term']}');")

    def select_all(self):
        return self.connection.execute("SELECT * FROM news_meta").fetchdf()

    def select_by_id(self, id):
        return self.connection.execute("SELECT * FROM news_meta WHERE id = ?", id).fetchdf()

    def select_by_title(self, title):
        return self.connection.execute("SELECT * FROM news_meta WHERE title = ?", title).fetchdf()

    def select_by_url(self, url):
        return self.connection.execute("SELECT * FROM news_meta WHERE url = ?", url).fetchdf()

    def select_by_term(self, source):
        self.connection.execute("PREPARE select_by_source AS "
                                "SELECT * FROM news_meta WHERE term = ?")
        return self.connection.execute(f"EXECUTE select_by_source('{source}');").fetchdf()

    def select_by_date(self, date_from, date_to):
        return self.connection.execute(f"SELECT * FROM news_meta WHERE strptime(timestamp, '%Y-%m-%dT%H:%M:%S%z') BETWEEN strptime('{date_from}','%Y-%m-%d') AND strptime('{date_to}','%Y-%m-%d')").fetchdf()

    def delete_all(self):
        self.connection.execute("DELETE FROM news_meta")

    def export(self, csv_file):
        self.connection.execute(f"EXPORT DATABASE '{csv_file}';")

    def close(self):
        self.connection.close()
        


In [4]:
def try_request(url, headers, params, isJson=True):

    # resp_proxy = requests.get('https://free-proxy-list.net/')
    # df = pd.read_html(resp_proxy.text)[0]
    # df_http = df[df['Https']=='no']
    # df_https = df[df['Https']=='yes']
    # proxy_http = f'https://{df_http["IP Address"].values[0]}:{df_http["Port"].values[0]}'
    # proxy_https = f'https://{df_https["IP Address"].values[0]}:{df_https["Port"].values[0]}'

    proxies={
        "http": 'socks5://193.35.18.30:30808',
        "https": 'socks5://193.35.18.30:30808'
    }

    response_page = requests.request("GET", url, headers=headers, params=params, timeout=5)
    try:
        response = response_page.json() if isJson else response_page.text

        if response_page.status_code > 300:
            print("Status code error: " + str(response_page.status_code))
            return False, params

        return True, response
    except requests.exceptions.JSONDecodeError as e:
        print(f"Bad Request: GET {url} \n Status Code: {response_page.status_code} | Error : {e}")
        return False, params
    except requests.exceptions.Timeout:
        print("Timed out")
        return False, params
    except requests.exceptions.RequestException as e:
        print(f"Bad Request: GET {url} \n")
        return False, params

In [5]:
newsMetaRepo = NewsMetaRepository("meta/bbc")

def create_request_url(term, page, page_size):
    template = 'https://web-cdn.api.bbci.co.uk/xd/search?terms={}?page={}?pageSize={}'
    url = template.format(term, page, page_size)
    return url

bbc_url = 'https://web-cdn.api.bbci.co.uk/xd/search'

def get_news_meta(search_term):
    hasData = True
    page = 0
    
    while hasData:
        print(f'Loading page {page}')

        response_ok, response = try_request(
            url = bbc_url,
            headers={},
            params = {
                'terms': search_term,
                'page': page,
                'pageSize': 100
            }
        )
                
        if response_ok:            
            if len(response["data"]) > 0:
                for data in response["data"]:
                    news_meta = {
                        'title': data['title'].replace("'","''"),
                        'term': search_term,
                        'timestamp': data['firstPublishedAt'], 
                        'url': data['path']
                    }
                    newsMetaRepo.insert(news_meta) 
            else: 
                print("Done loading.")
                hasData = False
        
        newsMetaRepo.export('meta/bbc')
        page += 1

get_news_meta("Tesla")

Loading page 0
Loading page 1
Loading page 2
Loading page 3
Loading page 4
Loading page 5
Loading page 6
Loading page 7
Loading page 8
Loading page 9
Loading page 10
Loading page 11
Loading page 12
Loading page 13
Loading page 14
Loading page 15
Done loading.


In [6]:
async def download_article(news_meta_url, news_meta_id, news_meta_filename):
    file_name = f'articles/bbc/{news_meta_filename}.txt'
    if not os.path.isfile(file_name):
        async with aiohttp.ClientSession() as session:
            print(f'Downloading {news_meta_id} , https://bbc.com{news_meta_url}')
    
            async with session.get(f'https://bbc.com{news_meta_url}') as response: 
                soup = BeautifulSoup(await response.text(), 'html.parser')
        
                complete_text = ''
        
                if soup.find('article') is not None:
                    text_blocks = soup.find('article').find_all('div', {'data-component':"text-block"})
                    for text_block in text_blocks:
                        text_paragraphs = text_block.find_all('p')
                        complete_text += ' '.join([paragraph.text for paragraph in text_paragraphs])
                else:
                    video_type = soup.find('div',{'data-testid':'video-page-video-section'})
                    if video_type is not None:
                        paragraphs = video_type.find_all('p')
                        complete_text += ' '.join([paragraph.text for paragraph in paragraphs])
                    
                with open(f'articles/bbc/{news_meta_filename}.txt', 'w') as f:
                    f.write(complete_text)


async def get_news_articles(source):
    news_meta_list = newsMetaRepo.select_by_term(source)
    # for i in range(0, 200): #len(news_meta_list)
    #     print(f'Downloading {news_meta_list["id"][i]} , https://bbc.com{news_meta_list["url"][i]}')
    #     download_article(news_meta_list['url'][i],news_meta_list['id'][i])
    #     sleep(0.35)

    def get_file_name(id):
        title = news_meta_list['title'][id].replace(' ','-').replace("/","-")
        return f"{title}-{parser.parse(news_meta_list['timestamp'][id]).timestamp()}"

    for i in range(0, len(news_meta_list))[::10]:
        print(i)
        try:
            task_list = [
                asyncio.create_task(
                    download_article(
                        news_meta_list['url'][i+idx],
                        news_meta_list['id'][i+idx],
                        get_file_name(i+idx)
                    )
                ) for idx in range(0,10 if len(news_meta_list) > i + 10 else len(news_meta_list) % 10)
            ]
            await asyncio.gather(*task_list)
        except Exception as e:
            print(f"ERROR DOWNLOADING ... RETRYING...")
            failed_tasks = [
                asyncio.create_task(
                    download_article(
                        news_meta_list['url'][i+idx],
                        news_meta_list['id'][i+idx],
                        get_file_name(i+idx)
                    )
                ) for idx in range(0,10 if len(news_meta_list) > i + 10 else len(news_meta_list) % 10)
            ]
            await asyncio.gather(*failed_tasks)

        if i % 50 == 0:
            sleep(3)

        sleep(0.300)
    # 
    # if len(failed_tasks) > 0:
    #     print(f'\n\nLength of the failed task:  {len(failed_tasks)}')
    #     # load failed tasks
    #     for i in range(0,len(failed_tasks))[::10]:
    #         print("Loading")
    #         await asyncio.gather(*(failed_tasks[i:i+10]))
    #         sleep(2)


In [7]:
company = "Tesla"

get_news_meta(company)
await get_news_articles(company)

Loading page 0
Loading page 1
Loading page 2
Loading page 3
Loading page 4
Loading page 5
Loading page 6
Loading page 7
Loading page 8
Loading page 9
Loading page 10
Loading page 11
Loading page 12
Loading page 13
Loading page 14
Loading page 15
Done loading.
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
